# Inception块
<img alt="image.png" height="600" src="/Users/loneker/Desktop/d2l/pic/iShot_2024-04-29_14.11.11.png" width="700"/>

> 3x3下面的1x1的卷积，输出通道数是96，目的是降低后面3x3的通道数，降低模型复杂度
> 白色的卷积层主要是为了改变通道数
> 蓝色的卷积块主要是来提取特征的
> 可能觉得3x3的卷积层能抽取到更多的信息，所以才让3x3的通道数占据了总通道数的一半

<img alt="image.png" height="600" src="/Users/loneker/Desktop/d2l/pic/iShot_2024-04-29_14.17.37.png" width="700"/>

# googLeNet
> 四个路径从不同层面抽取信息，然后在输出通道维合并
> 输入和输出等高宽，但通道数会变多

<img alt="image.png" height="800" src="/Users/loneker/Desktop/d2l/pic/iShot_2024-04-29_14.20.06.png" width="700"/>

# googLeNet Block 1 & 2
<img alt="image.png" height="800" src="/Users/loneker/Desktop/d2l/pic/iShot_2024-04-29_14.24.38.png" width="900"/>

> 1. 将通道数变多，将高宽减下去
> 2. 比AlexNet的更宽更高，使得能支持更深的网络

# googLeNet Block 3
<img alt="image.png" height="800" src="/Users/loneker/Desktop/d2l/pic/iShot_2024-04-29_14.29.44.png" width="1000"/>

> 1. 使用了两个Inception Block，经过两次Inception块之后再经一次MaxPooling将高宽减半
> 2. 主要通道数分配给3x3的卷积层，但不知道该层通道数与输入输出通道数之间的关系

# googLeNet Block 4 & 5
<img alt="image.png" height="800" src="/Users/loneker/Desktop/d2l/pic/iShot_2024-04-29_14.39.01.png" width="1000"/>

> 一张图片进入Net，输出是一个1024维的向量

# 变种
<img alt="image.png" height="700" src="/Users/loneker/Desktop/d2l/pic/iShot_2024-04-29_14.42.32.png" width="1000"/>

# 总结
1. Inception块用4条有不同超参数的卷积层和池化层的路来抽取不同的信息，主要优点是模型参数小，计算复杂度低
2. GoogleNet使用了9个Inception块，是第一个达到上百层的网络

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

class Inception(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        # 在通道维度上连结输出
        return torch.cat((p1, p2, p3, p4), dim=1)

# 实现每一个stage

In [2]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.ReLU(),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
b3 = nn.Sequential(Inception(192, 64, (96, 128), (16, 32), 32),
                   Inception(256, 128, (128, 192), (32, 96), 64),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64),
                   Inception(512, 160, (112, 224), (24, 64), 64),
                   Inception(512, 128, (128, 256), (24, 64), 64),
                   Inception(512, 112, (144, 288), (32, 64), 64),
                   Inception(528, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),
                   Inception(832, 384, (192, 384), (48, 128), 128),
                   nn.AdaptiveAvgPool2d((1,1)),
                   nn.Flatten())

net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(1024, 10))

# 为了训练更快，我们将输入的高和宽从224降低到96

In [3]:
X = torch.rand(size=(1, 1, 96, 96))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 24, 24])
Sequential output shape:	 torch.Size([1, 192, 12, 12])
Sequential output shape:	 torch.Size([1, 480, 6, 6])
Sequential output shape:	 torch.Size([1, 832, 3, 3])
Sequential output shape:	 torch.Size([1, 1024])
Linear output shape:	 torch.Size([1, 10])


In [ ]:
lr, num_epochs, batch_size = 0.1, 10, 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())